In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

In [3]:
import pandas as pd
import numpy as np
import pipit as pp

In [4]:
trace = pp.Trace.from_otf2("../../pipit/tests/data/ping-pong-otf2/")
pandas_df = trace.events[["Timestamp (ns)", "Name", "Event Type", "Process", "Thread"]].copy()
pandas_df["Process"] = pandas_df["Process"].astype(int)
idx = pandas_df.groupby(['Process']).cumcount()
pandas_df["idx"] = idx

In [5]:
pp.set_option("backend", "pandas")
pp.get_option("backend")

'pandas'

In [6]:
ds = pp.dsl2.Dataset()
ds.show()

Dataset (0 events)


In [7]:
# simulating a reader
row = pandas_df.iloc[0]
evt = pp.dsl2.Event(
    rank=row["Process"],
    idx=row["idx"],
    thread=row["Thread"],
    timestamp=row["Timestamp (ns)"],
    event_type=row["Event Type"],
    name=row["Name"],
)
evt

Event {'rank': 1, 'idx': 0, 'thread': 0, 'timestamp': 0.0, 'event_type': 'Instant', 'name': 'ProgramBegin'}

In [8]:
ds.push_event(evt)
ds.show()

Dataset (0 events)


In [9]:
ds.flush()
ds.show()

+--------+-------+----------+-------------+--------------+--------------+
|   rank |   idx |   thread |   timestamp | event_type   | name         |
|--------+-------+----------+-------------+--------------+--------------|
|      1 |     0 |        0 |           0 | Instant      | ProgramBegin |
+--------+-------+----------+-------------+--------------+--------------+
Dataset (1 event)


In [10]:
for i in range(1, len(pandas_df)):
    row = pandas_df.iloc[i]
    evt = pp.dsl2.Event(
        rank=row["Process"],
        idx=row["idx"],
        thread=row["Thread"],
        timestamp=row["Timestamp (ns)"],
        event_type=row["Event Type"],
        name=row["Name"],
    )
    ds.push_event(evt)

ds.flush()
ds.show()

+--------+-------+----------+--------------------+--------------+-----------------------+
| rank   | idx   | thread   | timestamp          | event_type   | name                  |
|--------+-------+----------+--------------------+--------------+-----------------------|
| 1      | 0     | 0        | 0.0                | Instant      | ProgramBegin          |
| 1      | 1     | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| 1      | 2     | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| 0      | 0     | 0        | 307730.9358165928  | Instant      | ProgramBegin          |
| 0      | 1     | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| ...    | ...   | ...      | ...                | ...          | ...                   |
| 1      | 57    | 0        | 199574793.63126454 | Leave        | MPI_Finalize          |
| 0      | 58    | 0        | 199575243.23094556 | Leave        | int main(int, char**) |
| 1      |

In [11]:
ds.show()

+--------+-------+----------+--------------------+--------------+-----------------------+
| rank   | idx   | thread   | timestamp          | event_type   | name                  |
|--------+-------+----------+--------------------+--------------+-----------------------|
| 1      | 0     | 0        | 0.0                | Instant      | ProgramBegin          |
| 1      | 1     | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| 1      | 2     | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| 0      | 0     | 0        | 307730.9358165928  | Instant      | ProgramBegin          |
| 0      | 1     | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| ...    | ...   | ...      | ...                | ...          | ...                   |
| 1      | 57    | 0        | 199574793.63126454 | Leave        | MPI_Finalize          |
| 0      | 58    | 0        | 199575243.23094556 | Leave        | int main(int, char**) |
| 1      |

In [12]:
ds.traces[0].show()

+-------+----------+--------------------+--------------+-----------------------+
| idx   | thread   | timestamp          | event_type   | name                  |
|-------+----------+--------------------+--------------+-----------------------|
| 0     | 0        | 307730.9358165928  | Instant      | ProgramBegin          |
| 1     | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| 2     | 0        | 346054.77444467926 | Enter        | MPI_Init              |
| 3     | 0        | 193643138.1741584  | Leave        | MPI_Init              |
| 4     | 0        | 193651646.20379105 | Enter        | MPI_Comm_size         |
| ...   | ...      | ...                | ...          | ...                   |
| 55    | 0        | 199320512.07918367 | Leave        | MPI_Recv              |
| 56    | 0        | 199514778.2785141  | Enter        | MPI_Finalize          |
| 57    | 0        | 199573648.15437022 | Leave        | MPI_Finalize          |
| 58    | 0        | 1995752

In [14]:
ds.traces[1].show()

+-------+----------+--------------------+--------------+-----------------------+
| idx   | thread   | timestamp          | event_type   | name                  |
|-------+----------+--------------------+--------------+-----------------------|
| 0     | 0        | 0.0                | Instant      | ProgramBegin          |
| 1     | 0        | 30083.086937435106 | Enter        | int main(int, char**) |
| 2     | 0        | 40288.33150186851  | Enter        | MPI_Init              |
| 3     | 0        | 193643835.4832178  | Leave        | MPI_Init              |
| 4     | 0        | 193652714.83827707 | Enter        | MPI_Comm_size         |
| ...   | ...      | ...                | ...          | ...                   |
| 55    | 0        | 199319910.70381415 | Leave        | MPI_Send              |
| 56    | 0        | 199529686.66029385 | Enter        | MPI_Finalize          |
| 57    | 0        | 199574793.63126454 | Leave        | MPI_Finalize          |
| 58    | 0        | 1995767

In [15]:
ds.head().show()

+--------+-------+----------+-------------+--------------+-----------------------+
|   rank |   idx |   thread |   timestamp | event_type   | name                  |
|--------+-------+----------+-------------+--------------+-----------------------|
|      1 |     0 |        0 |         0   | Instant      | ProgramBegin          |
|      1 |     1 |        0 |     30083.1 | Enter        | int main(int, char**) |
|      1 |     2 |        0 |     40288.3 | Enter        | MPI_Init              |
|      0 |     0 |        0 |    307731   | Instant      | ProgramBegin          |
|      0 |     1 |        0 |    336980   | Enter        | int main(int, char**) |
+--------+-------+----------+-------------+--------------+-----------------------+
Dataset (5 events)


In [16]:
ds.filter("timestamp > 1e5").show()

+--------+-------+----------+--------------------+--------------+-----------------------+
| rank   | idx   | thread   | timestamp          | event_type   | name                  |
|--------+-------+----------+--------------------+--------------+-----------------------|
| 0      | 0     | 0        | 307730.9358165928  | Instant      | ProgramBegin          |
| 0      | 1     | 0        | 336979.73374932166 | Enter        | int main(int, char**) |
| 0      | 2     | 0        | 346054.77444467926 | Enter        | MPI_Init              |
| 0      | 3     | 0        | 193643138.1741584  | Leave        | MPI_Init              |
| 1      | 3     | 0        | 193643835.4832178  | Leave        | MPI_Init              |
| ...    | ...   | ...      | ...                | ...          | ...                   |
| 1      | 57    | 0        | 199574793.63126454 | Leave        | MPI_Finalize          |
| 0      | 58    | 0        | 199575243.23094556 | Leave        | int main(int, char**) |
| 1      |

In [17]:
ds.loc[0, 28]

TypeError: 'NoneType' object is not subscriptable